Краулер для Проза ру,

авторы, рекомендованные редакцией

http://www.proza.ru/authors/editor.html

In [1]:
# Импортируем необходимые библиотеки:
import random
import time
import requests # http-запросы,
import re 
from bs4 import BeautifulSoup # удаление тегов html,
from tqdm import tqdm # красотуля для анализа прогресса

In [3]:
#список всех ссылок
hrefs = []
out = open(r"/home/mi_air/Downloads/spisok_proza_hrefs.txt","w", encoding="utf-8")

In [4]:
# Функия принимает адрес статьи на nplus1.ru и возвращает текст статьи и метаинформацию по ней.
def getArticleTextProza(adr):
    r = requests.get(adr) # Получаем текст статьи, ...
#print(r.text)
    tables=re.split("</div>",re.split('="tables"', r.text)[1])[0] # ... берем блок с метаинформацией, ...
# ... извлекаем из блока метаинформацию. 
    t1=re.split("</time>", re.split("<time", tables)[1])[0]
    n_time=re.split("</span>", re.split("<span>", t1)[1])[0]
    n_date=re.split("</span>", re.split("<span>", t1)[2])[0]
    n_rubr=re.split(">", re.split("</a>", re.split("<a href", tables)[1])[0])[1]
    n_diff=re.split("</span>", re.split('"difficult-value">', tables)[1])[0]
# Достаем заголовок, автора и текст статьи.
    n_head=re.split("</h1>",re.split('<h1>', r.text)[1])[0]
    n_author=re.split('" />',re.split('<meta name="author" content="', r.text)[1])[0]
    n_text=re.split("</div>", re.split("</figure>", re.split('</article>',re.split('<article', r.text)[1])[0])[1])[1]    

# "Откусываем" оставшиеся теги.
    beaux_text=BeautifulSoup(n_text, "lxml")
    n_text=beaux_text.get_text() 

# print(n_time, n_date, n_rubr, n_diff)
# print(n_head)
# print(n_author)
# print(n_text)
    return [n_time, n_date, n_rubr, n_diff, n_author, n_head, n_text]

# def getArticleTextNPlus1(adr):

In [5]:
# Получаем текст cо страницы, которая содержит ссылки на всех авторов, рекомендуемых сегодня

def getAuthors(link):
    r = requests.get(link)
    # Каждая ссылка на автора оформлена с помощью тега <li><a href="/avtor/
    refs=re.split('<li><a href="/avtor/', r.text)
    database=[]
    
    for i in refs[1:]:
        urlauthor = re.split(" class='recomlink'>",i)[0]
        urlauthor = "http://www.proza.ru/avtor/" + urlauthor
        urlauthor = urlauthor.strip('"')
        hrefs.append(urlauthor)
        out.write(urlauthor+"\n")
        print(urlauthor)
    #return(hrefs)




In [13]:
initial = "http://www.proza.ru/authors/editor.html" #первая ссылка,с кот.начинается обход
link1 = initial
while True:
    spisok = getAuthors(link1)
    s = requests.get(link1)
    refs="http://www.proza.ru/" + re.split('">', re.split('<p>Вы можете посмотреть список авторов, рекомендуемых редакцией <a href="', s.text)[1])[0]
    link1 = refs



NameError: name 'getAuthors' is not defined

In [ ]:
#необходимо проверить список на дубли
authors = open(r"/home/mi_air/Downloads/spisok_proza_hrefs.txt", "r", encoding="utf-8")
lst = []
for line in authors:
    lst.append(line)
lst = list(set(lst))


In [ ]:
out2 = open(r"/home/mi_air/Downloads/spisok_proza_hrefs_dedup.txt", "w", encoding="utf-8")
for item in lst:
    out2.write(item)

In [12]:
authortext = open(r"/home/mi_air/Downloads/proza_ru/spisok_proza_author_texts.txt", "w", encoding="utf-8")

In [3]:
#Теперь список авторов нам нужно превратить с список данных автора и ссылки на его тексты:
def getAuthorInfo(authorlink):
    r = requests.get(authorlink)
    author_texts_features = []
    try:
        author_name = re.split(" / Проза.ру</title>",re.split('<title>', r.text)[1])[0]
        author_items = re.split("</b>", re.split("Произведений: <b>", r.text)[1])[0]
        author_readeres = re.split("</b>", re.split("Читателей</a>: <b>", r.text)[1])[0]
        author_texts_features = []
        texts = re.split("<h2>Произведения</h2>", r.text)[1]
        texts2 = re.split('<li><a href="', texts)
        for line in texts2[1:]:
            if "poemlink" in line:
                textlink = "http://www.proza.ru" + re.split('" class',line)[0]
                textname = re.split("</a>", re.split('poemlink">', line)[1])[0]
                textdate = re.split(" ", re.split(", ", re.split("<small>- ",line)[1])[1])[0]
                textrubric = re.split(",", re.split("<small>- ", line)[1])[0]
                texttime = re.split(".\d{4} ",re.split("</small></li>", line)[0])[1]
                features = [author_name,author_items,author_readeres,textname,textrubric,textdate,texttime,textlink]
                author_texts_features.append(features)
                line = ""
                for i in features:
                    line += str(i)+"\t"
                line += "\n"
                authortext.write(line)
        
                
    except:
        pass

    return(author_texts_features )

    

In [14]:
def continuetexts(link):
    bl = False
    r = requests.get(link)
    if "продолжение:" in r.text:
        bl = True
    return(bl)

In [15]:
def books(link):
    bl = False
    r = requests.get(link)
    if "bookheader" in r.text:
        bl = True
    return(bl)

In [16]:
def bookitems(link):
    r = requests.get(link)
    lst = []
    for line in r.text:
        if '<div ID="bookheader"><a href="' in line:
            lst.append(line)
            
    for l in lst:
        l = "http://www.proza.ru" + re.split('<div ID="bookheader"><a href="', re.split('">', l)[0])[1]
        print(l)
    return(lst)

In [17]:
def continuelinks(link):
    r = requests.get(link)
    lst = []
    for line in r.text:
        if '&s=' in line:
            lst.append(line)
    for l in lst:
        l = "http://www.proza.ru" + re.split('<a href="', re.split('">', l)[0])[1]
    return (lst)

In [18]:
hrefs = open(r"/home/mi_air/Downloads/spisok_proza_hrefs_dedup.txt", "r", encoding="utf-8")
for link in hrefs:
    authorlink = link
    getAuthorInfo(authorlink)

    
    if books(link)==True:
        lst = bookitems(link)
        for item in lst:
            print(item)
            getAuthorInfo(item)
    if continuetexts(link) ==True:   
        lst = continuelinks(link)
        for item in lst:
            print(item)
            getAuthorInfo(item)

In [2]:
def getTextProza(textlink):
    r = requests.get(textlink)  
    text = re.split("</div>", re.split('<div class="text">', r.text)[1])[0]
# "Откусываем" оставшиеся теги.
    beaux_text=BeautifulSoup(text, "lxml")
    n_text=beaux_text.get_text() 
    return(n_text)

# def getArticleTextNPlus1(adr):

In [3]:
print(getTextProza('http://www.proza.ru/editor/2014/02/22/1843'))

Встреча первая.

Впервые я увидела Ульяну через приоткрытое окно больничной палаты. О том, что она лежит в станичной больнице, мне по секрету сообщила моя самая близкая подружка Нина, чья мама работала в больнице медсестрой.
Ульяна – это мама мальчика Ивана, который был восьмым и самым младшим  ребенком в большой крестьянской семье. С ним подружились мы с той самой осени 1960 года, когда он пришел с хутора Лебеди в 8 класс нашей станичной одиннадцатилетки. И дружба наша оказалась такой самозабвенной, будто мы родились в один день и час, крепко взявшись за руки, и только так были способны идти по жизни и познавать этот мир. Разомкни наши руки – и все исчезнет: и небо, и звезды, и порывистая река Кубань, и бескрайняя степь со всеми ее запахами, звуками и красками, и весь казачий люд, такой колоритный, зоркий, игривый и старомодный.
Но речь не о нас. Это совсем другая история.
Я стою на крыльце длинного одноэтажного здания больницы. Накрапает теплый дождь. Начало лета. По всему периметру 

In [ ]:
authortext = open(r"/home/mi_air/Downloads/proza_ru/spisok_proza_author_texts2.txt", "r", encoding="utf-8")
corpus = open(r"/home/mi_air/Downloads/proza_ru/corpus_proza_ru.txt", "w", encoding="utf-8")
for line in authortext:
    try:
        params = line.split("\t")
        #print(params[7])
        corpus.write("<author>"+params[0]+"\n")
        corpus.write("<authortexts>"+params[1]+"\n")
        corpus.write("<authorreaderes>"+params[2]+"\n")
        corpus.write("<textname>"+params[3]+"\n")
        corpus.write("<textrubric>"+params[4]+"\n")
        corpus.write("<date>"+params[5]+"\n")
        corpus.write("<time>"+params[6]+"\n")
        corpus.write("<source>"+params[7]+"\n")
        corpus.write("<textid>" + re.sub("/", "", re.split("//www.proza.ru/",params[7])[1]) + "\n")
        text = getTextProza(params[7])
        corpus.write("<text>"+"\n")
        corpus.write(text+"\n")
        corpus.write("</text>"+"\n")
        print(params[0])
    except:
        pass
    #time.sleep(random.uniform(1, 6))
        

Михаил Хаймович
Михаил Хаймович
Михаил Хаймович
Михаил Хаймович
Михаил Хаймович
Михаил Хаймович
Михаил Хаймович
Михаил Хаймович
Михаил Хаймович
Михаил Хаймович
Михаил Хаймович
Михаил Хаймович
Михаил Хаймович
Михаил Хаймович
Михаил Хаймович
Михаил Хаймович
Михаил Хаймович
Михаил Хаймович
Михаил Хаймович
Михаил Хаймович
Михаил Хаймович
Михаил Хаймович
Наталья Ковалёва
Галина Кузина
Галина Кузина
Галина Кузина
Галина Кузина
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Виктор Слободчиков
Викто